# Observe With Tracing

## Implement MLflow Tracing for observability and debugging

MLflow Tracing provides comprehensive observability for your GenAI applications, allowing you to:

- **Debug** issues by inspecting inputs, outputs, and intermediate steps
- **Collect** user feedback to improve your models
- **Optimize** latency by identifying bottlenecks

Implementing MLflow Tracing only requires _a single line of code_. You can attach end user feedback from your UI to the corresponding trace so you can quickly debug user-reported issues.

![tracing-demo](https://i.imgur.com/tNYUHdC.gif)


## Install packages (only required if running in a Databricks Notebook)

In [ ]:
%pip install -U -r ../../requirements.txt
dbutils.library.restartPython()

## Environment Setup

Load environment variables that connect to the MLflow server for trace logging and configure the sample application to access Databricks resources.


In [ ]:
import sys
sys.path.append('../')
sys.path.append('../../')

import os
from dotenv import load_dotenv
import mlflow
from mlflow_demo.utils import *

if mlflow.utils.databricks_utils.is_in_databricks_notebook():
  print("Running in Databricks Notebook")
  setup_databricks_notebook_env()
else:
  print("Running in Local IDE")
  setup_local_ide_env()

# Verify key variables are loaded
print('=== Environment Setup ===')
print(f'DATABRICKS_HOST: {os.getenv("DATABRICKS_HOST")}')
print(f'MLFLOW_EXPERIMENT_ID: {os.getenv("MLFLOW_EXPERIMENT_ID")}')
print(f'LLM_MODEL: {os.getenv("LLM_MODEL")}')
print(f'UC_CATALOG: {os.getenv("UC_CATALOG")}')
print(f'UC_SCHEMA: {os.getenv("UC_SCHEMA")}')
print('✅ Environment variables loaded successfully!')

import logging
logging.getLogger("urllib3").setLevel(logging.ERROR)
logging.getLogger("mlflow").setLevel(logging.ERROR)

# 🔍 Step 1: Instrument with Automatic and Manual Tracing

MLflow provides comprehensive observability for your GenAI applications with **just a single line of code**. Automatic tracing works out-of-the-box with 20+ popular frameworks including OpenAI, LangChain, Anthropic, and more. While automatic tracing handles most common cases, you can use manual tracing to capture your application's specific logic with custom spans, attributes, and detailed step-by-step tracking.

**📖 Code References**

- See `EmailGenerator` class definition: `mlflow_demo/agent/email_generator.py:53`
- Automatic tracing setup:
  - `mlflow_demo/agent/email_generator.py:173`
- Manual tracing with `@mlflow.trace`:
  - `mlflow_demo/agent/email_generator.py:210`
  - `mlflow_demo/agent/email_generator.py:252`
  - `mlflow_demo/agent/email_generator.py:388`
  - `mlflow_demo/agent/email_generator.py:590`
  - `mlflow_demo/agent/email_generator.py:632`

**📚 Documentation**

- **Automatic Tracing** - One-line setup for popular SDKs
- **Manual Tracing** - Custom spans and attributes

**▶️ Run the next cell to import the instrumented EmailGenerator class!**

In [ ]:
# Import our EmailGenerator class which uses MLflow Tracing
# Note: This import must come after loading environment variables
from mlflow_demo.agent.email_generator import EmailGenerator

print('✅ EmailGenerator imported successfully')

# 🚀 Step 2: Run the Email Application

Load sample customer data and generate an email using the instrumented EmailGenerator. This creates traces that you can view in the MLflow UI and attach feedback to later.

**▶️ Run the next cells to load customer data and generate your first traced email!**


In [ ]:
os.getcwd()

In [ ]:
import json
import os

if mlflow.utils.databricks_utils.is_in_databricks_notebook():
  input_data_path = '../data/input_data.jsonl'
else:
  # Fallback - try to find the file
  input_data_path = 'mlflow_demo/data/input_data.jsonl'

customers = []
with open(input_data_path, 'r') as f:
  for line in f:
    if line.strip():
      customers.append(json.loads(line))

print(f'📁 Loaded {len(customers)} customer records')
print('Available companies:')
for i, customer in enumerate(customers[:5]):  # Show first 5
  print(f'  {i + 1}. {customer["account"]["name"]} ({customer["account"]["industry"]})')
print(f'  ... and {len(customers) - 5} more')

# Let's use the first customer for our example
sample_customer = customers[0]
print(f'\n🎯 Selected customer: {sample_customer["account"]["name"]}')
print(f'Industry: {sample_customer["account"]["industry"]}')
print(f'Contact: {sample_customer["account"]["main_contact"]["name"]}')

: 

In [ ]:
# Initialize the EmailGenerator with our environment configuration
email_generator = EmailGenerator()

print('🔧 EmailGenerator initialized with:')
print(f'   Model: {email_generator.model}')
print(
  f'   Prompt: {email_generator.uc_catalog}.{email_generator.uc_schema}.{email_generator.prompt_name}@{email_generator.prompt_alias}'
)

# Get customer name and user instructions for the new API
customer_name = sample_customer['account']['name']
user_input = 'Please emphasize the upcoming product launch and schedule a demo for next week.'

print(f'\n📝 User instructions: {user_input}')
print(f'🎯 Customer: {customer_name}')
print('🚀 Generating email...')

# Generate the email - this will create MLflow traces!
result = email_generator.generate_email_with_retrieval(customer_name, user_input)

print('\n✅ Email generated successfully!')
print(f'🆔 Trace ID: {result.get("trace_id")}')
print(f'📧 Subject: {result.get("subject_line")}')
print(f'📄 Body preview: {result.get("body", "")[:200]}...')

# Store the trace_id for use in feedback logging
current_trace_id = result.get('trace_id')
print(f'\n💾 Stored trace_id for feedback: {current_trace_id}')

# Generate MLflow UI links for this trace
print('\n' + '=' * 60)
generate_trace_links(current_trace_id)
print('=' * 60)

# 👍 Step 3: Log User Feedback to Traces

Attach user feedback to the traces you just created. This links user satisfaction directly to specific model generations, making it easy to debug issues and improve your prompts.

**📖 Code References**

- Backend implementation `log_feedback()`: `mlflow_demo/agent/email_generator.py:304`
- Frontend implementation `handleFeedbackSubmit()`: `client/src/components/email-generator/EmailGenerator.tsx:183`

**📚 Documentation**

- [**Collecting User Feedback**](https://docs.databricks.com/aws/en/mlflow3/genai/tracing/collect-user-feedback/) - Link feedback to traces

**▶️ Run the next cells to submit both positive and negative feedback examples!**


In [ ]:
# Example 1: Submit positive feedback
if current_trace_id:
  positive_feedback = email_generator.log_feedback(
    trace_id=current_trace_id,
    value=True,  # True = positive feedback
    comment='Great email! The tone was perfect and it mentioned the product launch as requested.',
    user_name=sample_customer['sales_rep']['name'],
  )

  print('👍 Positive Feedback Submitted:')
  print(f'   Success: {positive_feedback["success"]}')
  print(f'   Message: {positive_feedback["message"]}')
  print(f'   Trace ID: {current_trace_id}')

  # Generate MLflow UI links to view the trace with feedback
  print('\n' + '=' * 60)
  print('📝 View the trace with your feedback attached:')
  generate_trace_links(current_trace_id)
  print('=' * 60)
else:
  print('⚠️ No trace_id available - please run the email generation cell first')

In [ ]:
# Let's generate another email with different customer data to show negative feedback
print('🔄 Generating another email for negative feedback example...')

# Use a different customer
second_customer = customers[1] if len(customers) > 1 else customers[0]
customer_name = second_customer['account']['name']
user_input = 'Make it very formal and include technical details about our platform.'

print(f'🎯 Customer: {customer_name}')
print(f'📝 Instructions: {user_input}')

# Generate second email
result2 = email_generator.generate_email_with_retrieval(customer_name, user_input)

# Get the trace_id - in your application, this is passed to the front end, which then passes it back to the backend when submitting feedback.
second_trace_id = result2.get('trace_id')

print('✅ Second email generated')
print(f'🆔 Trace ID: {second_trace_id}')

# Submit negative feedback
if second_trace_id:
  negative_feedback = email_generator.log_feedback(
    trace_id=second_trace_id,
    value=False,  # False = negative feedback
    comment="The email was too generic and didn't capture our company's specific needs. Could be more personalized.",
    user_name=second_customer['sales_rep']['name'],
  )

  print('\n👎 Negative Feedback Submitted:')
  print(f'   Success: {negative_feedback["success"]}')
  print(f'   Message: {negative_feedback["message"]}')
  print(f'   Trace ID: {second_trace_id}')

  # Generate MLflow UI links to view the trace with negative feedback
  print('\n' + '=' * 60)
  print('📝 View the trace with negative feedback attached:')
  generate_trace_links(second_trace_id)
  print('=' * 60)

# 🎯 Summary and Next Steps

Congratulations! You've successfully implemented MLflow tracing for GenAI applications with automatic instrumentation, user feedback collection, and custom span creation.

## What You've Accomplished

✅ **Enabled automatic tracing** with `mlflow.openai.autolog()`  
✅ **Generated traced emails** using instrumented code with `@mlflow.trace`  
✅ **Attached user feedback** to traces with `mlflow.log_feedback()`  
✅ **Created custom spans** with detailed attributes and metrics

**Next steps**

- [Follow the quickstart to instrument your own app](https://docs.databricks.com/aws/en/mlflow3/genai/getting-started/tracing/)
- [Read the tracing documentation](https://docs.databricks.com/aws/en/mlflow3/genai/tracing/)
